In [18]:
import pandas as pd
import numpy as np
from prophet import Prophet

# Assume your dataframe is already loaded as `merged_df`
# and contains monthly values or is interpolated from annual values

# Pivot and interpolate to get monthly educator counts
merged_df['REF_DATE'] = pd.to_datetime(merged_df['REF_DATE'], format='%d-%m-%Y')
educator_df = merged_df[['REF_DATE', 'Province', 'Total, work status']].copy()
pivot_df = educator_df.pivot(index='REF_DATE', columns='Province', values='Total, work status')
monthly_df = pivot_df.resample('MS').interpolate(method='linear')

# Prepare to store all forecasts
forecast_all = []

# List of all provinces and Canada
geo_list = monthly_df.columns.tolist()

for geo in geo_list:
    df_geo = monthly_df[[geo]].reset_index()
    df_geo.columns = ['ds', 'y']

    # Remove outliers using IQR
    q1 = df_geo['y'].quantile(0.25)
    q3 = df_geo['y'].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    df_geo = df_geo[(df_geo['y'] >= lower_bound) & (df_geo['y'] <= upper_bound)]

    if len(df_geo) < 10:
        continue

    # Fit Prophet model
    model = Prophet(
        yearly_seasonality=False,
        weekly_seasonality=False,
        daily_seasonality=False,
        changepoint_prior_scale=0.5
    )
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    model.add_seasonality(name='yearly', period=365.25, fourier_order=10)
    model.fit(df_geo)

    # Create future dataframe from Jan 2024 to Dec 2035
    future_months = pd.date_range(start='2024-01-01', end='2035-12-01', freq='MS')
    future_df = pd.DataFrame({'ds': future_months})

    # Forecast
    forecast = model.predict(future_df)

    # Format results
    result = forecast[['ds', 'yhat']].copy()
    result.columns = ['REF_DATE', 'Total Number of Educators']
    result['GEO'] = geo

    forecast_all.append(result)

# Combine all forecasts
final_forecast_df = pd.concat(forecast_all, ignore_index=True)
final_forecast_df['Total Number of Educators'] = final_forecast_df['Total Number of Educators'].round().astype(int)
final_forecast_df['REF_DATE'] = final_forecast_df['REF_DATE'].dt.strftime('01-%m-%Y')

# Sort and export
final_forecast_df = final_forecast_df.sort_values(by=['REF_DATE', 'GEO']).reset_index(drop=True)
final_forecast_df.to_csv("Number_of_Educators__Forecast_2024_2035.csv", index=False)

# Preview
print(final_forecast_df.head())
print("✅ Forecast dataset saved to 'Number_of_Educators__Forecast_2024_2035.csv'")


01:38:08 - cmdstanpy - INFO - Chain [1] start processing
01:38:08 - cmdstanpy - INFO - Chain [1] done processing
01:38:08 - cmdstanpy - INFO - Chain [1] start processing
01:38:08 - cmdstanpy - INFO - Chain [1] done processing
01:38:08 - cmdstanpy - INFO - Chain [1] start processing
01:38:09 - cmdstanpy - INFO - Chain [1] done processing
01:38:09 - cmdstanpy - INFO - Chain [1] start processing
01:38:09 - cmdstanpy - INFO - Chain [1] done processing
01:38:09 - cmdstanpy - INFO - Chain [1] start processing
01:38:09 - cmdstanpy - INFO - Chain [1] done processing
01:38:10 - cmdstanpy - INFO - Chain [1] start processing
01:38:10 - cmdstanpy - INFO - Chain [1] done processing
01:38:10 - cmdstanpy - INFO - Chain [1] start processing
01:38:10 - cmdstanpy - INFO - Chain [1] done processing
01:38:10 - cmdstanpy - INFO - Chain [1] start processing
01:38:10 - cmdstanpy - INFO - Chain [1] done processing
01:38:11 - cmdstanpy - INFO - Chain [1] start processing
01:38:11 - cmdstanpy - INFO - Chain [1]

     REF_DATE  Total Number of Educators               GEO
0  01-01-2024                      40515           Alberta
1  01-01-2024                      53427  British Columbia
2  01-01-2024                     431262            Canada
3  01-01-2024                      13790          Manitoba
4  01-01-2024                       8243     New Brunswick
✅ Forecast dataset saved to 'Number_of_Educators__Forecast_2024_2035.csv'


In [19]:
edu_df.head()

,REF_DATE,Province,Full-time educators,Part-time educators,"Total, work status",Education price index (EPI),Fees and contractual services sub-index,Instructional supplies sub-index,Non-salary sub-index,Non-teaching salaries sub-index,...,Total operating expenditures,College,Elementary and/or High School,University,Educator_to_OperatingSpending,Salary_to_EPI,OpSpend_per_Educator,Education_Access_Index,Capital_Efficiency,Year
484,01-01-1991,Quebec,68160.681818,33531.545455,101692.090909,125.03,136.25,130.64,131.11,121.90,...,578801.0,21.413793,5.551724,21.655172,0.175694,2559.617692,5.691701,16.206897,4833.292919,1991
485,01-01-1991,Prince Edward Island,1462.363636,200.590909,1662.136364,128.12,135.35,130.64,122.65,128.26,...,489735.5,10.034483,5.413793,26.620690,0.003394,2101.838121,294.642191,14.022989,3955.440334,1991
486,01-01-1991,Canada,307115.863636,80013.136364,387129.136364,126.27,136.11,130.64,127.69,121.21,...,1343906.0,14.517241,6.758621,24.793103,0.288063,5956.094084,3.471467,15.356322,11220.992728,1991
487,01-01-1991,Nova Scotia,9786.954545,15979.881818,9787.500000,122.40,138.06,130.64,125.75,120.87,...,311969.5,9.172414,6.689655,26.689655,0.031373,1392.757353,31.874278,14.183908,2711.026219,1991
488,01-01-1991,Saskatchewan,10467.409091,1696.636364,12164.181818,119.44,134.61,130.64,126.33,119.14,...,13701.0,6.137931,6.517241,24.793103,0.887832,49.497656,1.126340,12.482759,122.385384,1991


In [20]:
merged_df.head(20
            )

,REF_DATE,Province,Full-time educators,Part-time educators,"Total, work status",Education price index (EPI),Fees and contractual services sub-index,Instructional supplies sub-index,Non-salary sub-index,Non-teaching salaries sub-index,...,Year,Total PRs,Total TRs,Total Births,Total Deaths,Population Estimate,Educators_per_1000,OpSpend_per_Capita,Access_Index_per_Capita,Data_Quality_Flag
0,1991-01-01,Quebec,68160.681818,33531.545455,101692.090909,125.03,136.25,130.64,131.11,121.90,...,1991,0.0,0.0,581654.0,98242.0,7026241.0,14.47,0.082377,2.306624e-06,False
1,1991-01-01,Prince Edward Island,1462.363636,200.590909,1662.136364,128.12,135.35,130.64,122.65,128.26,...,1991,0.0,0.0,11296.0,2376.0,130477.0,12.74,3.753424,1.074748e-04,False
2,1991-01-01,Canada,307115.863636,80013.136364,387129.136364,126.27,136.11,130.64,127.69,121.21,...,1991,0.0,0.0,2413304.0,391138.0,27790000.0,13.93,0.048359,5.525844e-07,False
3,1991-01-01,Nova Scotia,9786.954545,15979.881818,9787.500000,122.40,138.06,130.64,125.75,120.87,...,1991,0.0,0.0,72112.0,14510.0,912792.0,10.72,0.341775,1.553904e-05,False
4,1991-01-01,Saskatchewan,10467.409091,1696.636364,12164.181818,119.44,134.61,130.64,126.33,119.14,...,1991,0.0,0.0,91862.0,16196.0,1002651.0,12.13,0.013665,1.244975e-05,False
5,1991-01-01,British Columbia,29352.818182,8329.909091,37682.454545,128.46,128.43,130.64,119.38,120.16,...,1991,0.0,0.0,273628.0,47954.0,3339935.0,11.28,0.064098,4.263969e-06,False
6,1991-01-01,Newfoundland and Labrador,5406.272727,1131.545455,6538.772727,126.44,136.73,130.64,126.91,128.26,...,1991,0.0,0.0,42968.0,7596.0,577377.0,11.32,0.386695,2.558141e-05,False
7,1991-01-01,New Brunswick,7382.318182,317.454545,7699.772727,122.23,137.62,130.64,122.70,120.87,...,1991,0.0,0.0,56978.0,10938.0,743210.0,10.36,0.181064,1.716691e-05,False
8,1991-01-01,Manitoba,12008.045455,1908.000000,13916.590909,122.47,136.62,130.64,124.74,120.95,...,1991,0.0,0.0,103642.0,17886.0,1106196.0,12.58,0.041465,1.157534e-05,False
9,1991-01-01,Alberta,31970.454545,8851.636364,40822.227273,120.70,135.07,130.64,125.78,120.18,...,1991,0.0,0.0,256630.0,28902.0,2572947.0,15.87,0.030589,4.726456e-06,False


In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt

# Prepare to store all forecasts
forecast_all = []

# Get province list + 'Canada' for national forecast
geo_list = merged_df['Province'].unique().tolist()
geo_list.append('Canada')

# Loop through each province (and Canada)
for geo in geo_list:
    if geo == 'Canada':
        df_geo = merged_df.groupby('Year')['Total, work status'].mean().reset_index()
    else:
        df_geo = merged_df[merged_df['Province'] == geo].groupby('Year')['Total, work status'].mean().reset_index()

    # Ensure column names match Prophet requirements
    df_geo.columns = ['ds', 'y']
    df_geo['ds'] = pd.to_datetime(df_geo['ds'], format='%Y')

    # Remove outliers
    q1 = df_geo['y'].quantile(0.25)
    q3 = df_geo['y'].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    df_geo = df_geo[(df_geo['y'] >= lower_bound) & (df_geo['y'] <= upper_bound)]

    # Skip if insufficient data
    if len(df_geo) < 10:
        continue

    # Fit Prophet model
    model = Prophet(
        yearly_seasonality=False,
        daily_seasonality=False,
        weekly_seasonality=False,
        changepoint_prior_scale=0.5  # Increase flexibility
    )
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    model.add_seasonality(name='yearly', period=365.25, fourier_order=10)
    model.fit(df_geo)

    # Create monthly future dataframe (from Jan 2024 to Dec 2035)
    future_months = pd.date_range(start='2024-01-01', end='2035-12-01', freq='MS')
    future_df = pd.DataFrame({'ds': future_months})

    # Predict
    forecast = model.predict(future_df)

    # Extract relevant columns
    result = forecast[['ds', 'yhat']].copy()
    result.columns = ['REF_DATE', 'Total, work status']
    result['GEO'] = geo

    # Store result
    forecast_all.append(result)

# Combine all province forecasts
final_forecast_df = pd.concat(forecast_all, ignore_index=True)
final_forecast_df = final_forecast_df.rename(columns={'Total, work status': 'Total Number of Educators'})

# Sort by REF_DATE and GEO
final_forecast_df = final_forecast_df.sort_values(by=['REF_DATE', 'GEO']).reset_index(drop=True)

# Optional: Save to CSV
final_forecast_df.to_csv("Number_of_Educators__Forecast_2024_2035.csv", index=False)

# Display preview
print(final_forecast_df.head())
print("✅ Forecast dataset saved to 'Number_of_Educators__Forecast_2024_2035.csv'")

01:38:12 - cmdstanpy - INFO - Chain [1] start processing
01:38:12 - cmdstanpy - INFO - Chain [1] done processing
01:38:12 - cmdstanpy - INFO - Chain [1] start processing
01:38:27 - cmdstanpy - INFO - Chain [1] done processing
01:38:27 - cmdstanpy - INFO - Chain [1] start processing
01:38:44 - cmdstanpy - INFO - Chain [1] done processing
01:38:44 - cmdstanpy - INFO - Chain [1] start processing
01:38:58 - cmdstanpy - INFO - Chain [1] done processing
01:38:59 - cmdstanpy - INFO - Chain [1] start processing
01:38:59 - cmdstanpy - INFO - Chain [1] done processing
01:38:59 - cmdstanpy - INFO - Chain [1] start processing
01:38:59 - cmdstanpy - INFO - Chain [1] done processing
01:38:59 - cmdstanpy - INFO - Chain [1] start processing
01:38:59 - cmdstanpy - INFO - Chain [1] done processing
01:39:00 - cmdstanpy - INFO - Chain [1] start processing
01:39:15 - cmdstanpy - INFO - Chain [1] done processing
01:39:15 - cmdstanpy - INFO - Chain [1] start processing
01:39:15 - cmdstanpy - INFO - Chain [1]